In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
#python setup_database.py & python setup_qdrant.py & python setup_elastic.py & python setup_redis.py

In [3]:
import sys
sys.path.append('/app')

In [4]:

from api_client import APIClient
from uuid import UUID
import asyncio
from datetime import datetime, timedelta

In [5]:

client = await APIClient.create(email="2280905@gmail.com")

In [6]:
client

In [7]:

# Auth examples
#auth_result = await client.google_auth("google_token_here", {"utm_source": "web"})



In [8]:
token_result = await client.submit_token("3ae04e20124d40babc5107e658c666b6")
    


In [9]:
token_result

{'user_id': 'ef7c085b-fdb5-4c94-b7b6-a61a3d04c210',
 'user_name': 'Dmitriy Grankin',
 'image': None}

In [10]:
# Content management
# Get contents with filters
contents = await client.get_contents(
    content_type="meeting",
    filters=[{
       "type": "speaker",
       "values": ["Dmitry Grankin"]
    }],
    offset=0,
    limit=20
)

contents



#failed - filter does not work

{'total': 778,
 'contents': [{'content_id': '005d79ed-bbf8-44d3-8a21-ea859fe47ce5',
   'type': 'meeting',
   'text': None,
   'timestamp': '2024-06-05T13:53:15.818873',
   'is_indexed': False,
   'is_owner': True,
   'access_level': 'owner',
   'entities': []},
  {'content_id': '00a949d7-cdb8-4988-a138-0a37491ff177',
   'type': 'meeting',
   'text': None,
   'timestamp': '2024-10-17T11:06:26',
   'is_indexed': False,
   'is_owner': True,
   'access_level': 'owner',
   'entities': []},
  {'content_id': '01266938-c09c-4354-a084-1c4c9b36ac37',
   'type': 'meeting',
   'text': None,
   'timestamp': '2024-07-31T19:05:41.579511',
   'is_indexed': False,
   'is_owner': True,
   'access_level': 'owner',
   'entities': [{'name': 'Avalanche Lion', 'type': 'speaker'}]},
  {'content_id': '0179e794-e071-45e2-ae1c-f9d3880dc789',
   'type': 'meeting',
   'text': None,
   'timestamp': '2024-06-30T07:03:32.394896',
   'is_indexed': False,
   'is_owner': True,
   'access_level': 'owner',
   'entities': 

In [11]:
# Add new content
content = await client.add_content(
    body="Sample content text",
    content_type="note",
    entities=[{"type": "speaker", "name": "John Doe"}]
)


In [12]:
content

{'content_id': 'b97dc4ad-b5d3-4a5a-a7e5-b88ab95a559a'}

In [13]:
content_id = UUID(content["content_id"])

# Get single content
single_content = await client.get_content(content_id)
single_content

In [15]:
# Modify content
modified = await client.modify_content(
    content_id,
    body="Updated content text",
    entities=[{"type": "speaker", "name": "Jane Doe"}]
)


In [16]:
modified

{'success': True}

In [17]:
# Get single content
single_content = await client.get_content(content_id)
single_content



{'content_id': 'b97dc4ad-b5d3-4a5a-a7e5-b88ab95a559a',
 'type': 'note',
 'text': 'Updated content text',
 'timestamp': '2025-01-28T17:03:51.724949',
 'is_indexed': False,
 'access_level': 'owner',
 'is_owner': True,
 'entities': [{'name': 'Jane Doe', 'type': 'speaker'}],
 'parent_id': None,
 'meta': None}

In [18]:
archived = await client.archive_content(content_id)

In [19]:
archived

{'success': True}

In [20]:
content_id

UUID('b97dc4ad-b5d3-4a5a-a7e5-b88ab95a559a')

In [21]:
single_content = await client.get_content(content_id)
single_content

#OK - content unavailable after archive

ValueError: Content not found or no access

In [22]:
restored = await client.restore_content(content_id)
single_content = await client.get_content(content_id)
single_content

{'content_id': 'b97dc4ad-b5d3-4a5a-a7e5-b88ab95a559a',
 'type': 'note',
 'text': 'Updated content text',
 'timestamp': '2025-01-28T17:03:51.724949',
 'is_indexed': False,
 'access_level': 'owner',
 'is_owner': True,
 'entities': [{'name': 'Jane Doe', 'type': 'speaker'}],
 'parent_id': None,
 'meta': None}

In [23]:
deleted = await client.delete_content(content_id)
single_content = await client.get_content(content_id)
single_content
    


ValueError: Content not found or no access

In [24]:
restored = await client.restore_content(content_id)
single_content = await client.get_content(content_id)
single_content

#OK - content unavailable after delete

ValueError: Content not found or not archived

In [25]:
# Add new content
content = await client.add_content(
    body="Sample content text",
    content_type="note",
    entities=[{"type": "speaker", "name": "John Doe"}]
)
content

{'content_id': 'a7ffdc5c-351c-4c3c-8f32-b15301cfcb9d'}

In [29]:
# Sharing
share_link = await client.create_share_link(
    access_level="transcript",
    meeting_ids=[content_id],
    target_email="recipient@example.com",
    expiration_hours=24
)
    


APIError: Failed to create share link: Internal Server Error

In [ ]:
    accepted = await client.accept_share_link(
        token=share_link["token"],
        accepting_email="recipient@example.com"
    )
    
    # Index content
    indexed = await client.index_content(content_id)
    


In [31]:
# Entity management
entities = await client.get_entities("speaker", offset=0, limit=20)
entities


[{'name': 'John Doe', 'id': 5704, 'type': 'speaker'},
 {'name': 'Jane Doe', 'id': 5705, 'type': 'speaker'},
 {'name': 'Arina Soboleva', 'id': 3665, 'type': 'speaker'},
 {'name': 'Dmitry Grankin', 'id': 3448, 'type': 'speaker'},
 {'name': 'Artem Puzik', 'id': 3484, 'type': 'speaker'},
 {'name': 'Sergey Ryabenko', 'id': 3441, 'type': 'speaker'},
 {'name': 'Dmitriy Grankin', 'id': 3440, 'type': 'speaker'},
 {'name': 'Pedro Teixeira', 'id': 4131, 'type': 'speaker'},
 {'name': 'Mariana Montenegro', 'id': 5432, 'type': 'speaker'},
 {'name': 'Alex Shevliakov', 'id': 3491, 'type': 'speaker'},
 {'name': 'Graham Ó Maonaigh', 'id': 5197, 'type': 'speaker'},
 {'name': 'Matt Lewis', 'id': 3543, 'type': 'speaker'}]

In [32]:
# Entity management
entities = await client.get_entities("tag", offset=0, limit=20)
entities


[]

In [33]:
# Chat interactions
async for response in client.chat(
    query="What is this document about?",
    content_id=content_id,
    model="gpt-4o-mini",
    temperature=0.7
):
    print(response["content"])



APIError: Chat request failed: {"detail":"Not Found"}

In [ ]:
    # Edit chat message
    async for response in client.edit_chat_message(
        thread_id="thread_123",
        message_index=1,
        new_content="Updated message content",
        model="gpt-4o-mini"
    ):
        print(response["content"])
    
    # Thread management
    threads = await client.get_threads(
        content_id=content_id,
        only_archived=False,
        limit=50,
        start_date=datetime.now() - timedelta(days=7)
    )
    
    thread = await client.get_thread("thread_123")
    archived_thread = await client.archive_thread("thread_123")
    restored_thread = await client.restore_thread("thread_123")
    renamed = await client.rename_thread("thread_123", "New Thread Name")

# Run examples
asyncio.run(main())